In [1]:
import os

In [2]:
%pwd

'e:\\DEEP LEARNING\\Language-Translation-Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\DEEP LEARNING\\Language-Translation-Project'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [19]:
from LangTranslation.constants import *
from LangTranslation.utils.common import read_yaml , create_directories


In [47]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [48]:
import os
import urllib.request as request
import zipfile
from LangTranslation.logging import logger
from LangTranslation.utils.common import get_size
import shutil


In [55]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [56]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    Data_ingestion=DataIngestion(config=data_ingestion_config)
    Data_ingestion.download_file()
    Data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-19 15:15:14,567: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-19 15:15:14,572: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-19 15:15:14,575: INFO: common: created directory at: artifacts]
[2024-05-19 15:15:14,579: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-19 15:15:17,223: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 343230
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "7a036b8e8f968c65f6f64a19c67914ddb33f3545b0a3610bd90b278ee91487d1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2590:0862:375C61:4256CF:6649CA29
Accept-Ranges: bytes
Date: Sun, 19 May 2024 09:45:14 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4750-BOM
X-Cache: MISS
